In [42]:
import pandas as pd
import numpy as np

In [2]:
df_NWHL = pd.read_csv(f"Rotman MMA Summer Datathon NWHL.csv")
df_NWHL.head()

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,...,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2021-01-23,Minnesota Whitecaps,Boston Pride,1,20:00,5,5,0,0,Boston Pride,...,Faceoff Win,100,43,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN
1,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:58,5,5,0,0,Boston Pride,...,Puck Recovery,107,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:57,5,5,0,0,Boston Pride,...,Zone Entry,125,28,Carried,NaN,NaN,NaN,Maddie Rowe,NaN,NaN
3,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:55,5,5,0,0,Boston Pride,...,Shot,131,28,Snapshot,On Net,t,f,NaN,NaN,NaN
4,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:53,5,5,0,0,Boston Pride,...,Faceoff Win,169,21,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN


In [3]:
df_NWHL.columns

Index(['game_date', 'Home Team', 'Away Team', 'Period', 'Clock',
       'Home Team Skaters', 'Away Team Skaters', 'Home Team Goals',
       'Away Team Goals', 'Team', 'Player', 'Event', 'X Coordinate',
       'Y Coordinate', 'Detail 1', 'Detail 2', 'Detail 3', 'Detail 4',
       'Player 2', 'X Coordinate 2', 'Y Coordinate 2'],
      dtype='object')

In [4]:
df_player_list = df_NWHL.groupby(["Team", "Player"]).size().reset_index().rename(columns={0:"count"})
player_list = df_player_list.Player.tolist()

In [5]:
df_player_list.head()

,Team,Player,count
0,Boston Pride,Briana Mastel,244
1,Boston Pride,Carlee Turner,251
2,Boston Pride,Christina Putigna,365
3,Boston Pride,Jenna Rheault,73
4,Boston Pride,Jillian Dempsey,336


In [6]:
df_player_list["Player"].value_counts().unique()

array([1], dtype=int64)

In [7]:
#df.shape[0]

In [8]:
def shooting_stats(player_name, df):
    df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
    not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()
    df_shot = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Shot') & (x['Player']==player_name)]['Event'].count()).to_frame(name='shot_number')
    df_shot = df_shot.drop(not_played_dates)
    df_goal = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Goal') & (x['Player']==player_name)]['Event'].count()).to_frame(name='goal_number')
    df_goal = df_goal.drop(not_played_dates)
    df_scoring_merged = df_shot.join(df_goal, how="left")
    num_games_played = len(df_scoring_merged)
    df_scoring_merged["total_shot_number"] = df_scoring_merged["shot_number"] + df_scoring_merged["goal_number"]
    df_scoring_merged["shot_percentage"] = df_scoring_merged["goal_number"]/df_scoring_merged["total_shot_number"]
    
    total_shots_made = df_scoring_merged["total_shot_number"].sum()
    total_goals_made = df_scoring_merged["goal_number"].sum()
    average_shots_made = df_scoring_merged["total_shot_number"].mean()
    average_shots_percentage = df_scoring_merged["shot_percentage"].mean()
    overall_shots_percentage = (df_scoring_merged["goal_number"].sum())/(df_scoring_merged["total_shot_number"].sum())
    
    player_shooting_stats_dict = {
        "name": player_name,
        "num_games_played": num_games_played,
        "total_shots_made": total_shots_made,
        "total_goals_made": total_goals_made,
        "average_shots_made": average_shots_made,
        "average_shots_percentage": average_shots_percentage,
        "overall_shots_percentage": overall_shots_percentage
    }
    
    return player_shooting_stats_dict

In [9]:
len(player_list)

126

In [10]:
def passing_stats(player_name, df):
    df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
    not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()
    df_play = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Play') & (x['Player']==player_name)]['Event'].count()).to_frame(name='play_number')
    df_play = df_play.drop(not_played_dates)
    df_incom_play = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Incomplete Play') & (x['Player']==player_name)]['Event'].count()).to_frame(name='incom_play_number')
    df_incom_play = df_incom_play.drop(not_played_dates)
    df_passing_merged = df_incom_play.join(df_play, how="left")
    
    num_games_played = len(df_passing_merged)
    
    df_passing_merged["total_pass_number"] = df_passing_merged["play_number"] + df_passing_merged["incom_play_number"]
    df_passing_merged["pass_percentage"] = df_passing_merged["play_number"]/df_passing_merged["total_pass_number"]
    
    total_passes_made = df_passing_merged["total_pass_number"].sum()
    total_plays_made = df_passing_merged["play_number"].sum()
    average_passes_made = df_passing_merged["total_pass_number"].mean()
    average_passes_percentage = df_passing_merged["pass_percentage"].mean()
    overall_passes_percentage = (df_passing_merged["play_number"].sum())/(df_passing_merged["total_pass_number"].sum())
    

    
    assist_dict = dict()
    df_len = len(df)

    for i in range(len(df)):
        if i == df_len-1:
            pass
        elif (df.loc[i, "Player"] == player_name) & (df.loc[i, "Event"] == "Play"):
            if (df.loc[i+1, "Event"] == "Goal") & (df.loc[i, "Team"] == df.loc[i+1, "Team"]):
                assist_dict[str(df.loc[i, "game_date"]) + "_" + str(df.loc[i, "Clock"]) + "_" + str(df.loc[i+1, "Player"])] = "passer " + df.loc[i, "Player"]
    
    total_assists = len(assist_dict)
    
    player_passing_stats_dict = {
        "name": player_name,
        "num_games_played": num_games_played,
        "total_passes_made": total_passes_made,
        "total_plays_made": total_plays_made,
        "average_passes_made": average_passes_made,
        "average_passes_percentage": average_passes_percentage,
        "overall_passes_percentage": overall_passes_percentage,
        "total_assists": total_assists
    }
    return player_passing_stats_dict

In [11]:
def dumping_stats(player_name, df):
    df_len = len(df)

    df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
    not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()


    dump_dict = dict()

    for i in range(len(df)):
        game_date = df.loc[i, "game_date"]
        if game_date in not_played_dates:
            pass
        elif game_date in dump_dict:
            if (df.loc[i, 'Event']=='Dump In/Out') & (df.loc[i, 'Player']==player_name):
                if 75 < df.loc[i, 'X Coordinate'] < 125:
                    if df.loc[i, "Detail 1"] == 'Retained':
                        dump_dict[game_date]["number_of_retained_dump_in"] += 1
                    elif df.loc[i, "Detail 1"] == 'Lost':
                        dump_dict[game_date]["number_of_lost_dump_in"] += 1
                elif df.loc[i, 'X Coordinate'] < 75:
                    if df.loc[i, "Detail 1"] == 'Retained':
                        dump_dict[game_date]["number_of_retained_dump_out"] += 1
                    elif df.loc[i, "Detail 1"] == 'Lost':
                        dump_dict[game_date]["number_of_lost_dump_out"] += 1

        elif game_date not in dump_dict:
            dump_dict[game_date] = dict()
            dump_dict[game_date]["number_of_retained_dump_in"] = 0
            dump_dict[game_date]["number_of_lost_dump_in"] = 0
            dump_dict[game_date]["number_of_retained_dump_out"] = 0
            dump_dict[game_date]["number_of_lost_dump_out"] = 0

    df_dumping = pd.DataFrame.from_dict(dump_dict).T
    num_games_played = len(df_dumping)

    total_dump_ins_made = df_dumping["number_of_retained_dump_in"].sum() + df_dumping["number_of_lost_dump_in"].sum()
    total_dump_outs_made = df_dumping["number_of_retained_dump_out"].sum() + df_dumping["number_of_lost_dump_out"].sum()

    average_dump_ins_made = (df_dumping["number_of_retained_dump_in"] + df_dumping["number_of_lost_dump_in"]).mean()
    average_dump_outs_made = (df_dumping["number_of_retained_dump_out"] + df_dumping["number_of_lost_dump_out"]).mean()

    average_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"]/(df_dumping["number_of_retained_dump_in"] + df_dumping["number_of_lost_dump_in"])).mean()
    average_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"]/(df_dumping["number_of_retained_dump_out"] + df_dumping["number_of_lost_dump_out"])).mean()

    overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
    overall_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"].sum())/(df_dumping["number_of_lost_dump_out"].sum())

    player_passing_stats_dict = {
        "name": player_name,
        "num_games_played": num_games_played,
        "total_dump_ins_made": total_dump_ins_made,
        "average_dump_ins_made": average_dump_ins_made,
        "average_dump_ins_percentage": average_dump_ins_percentage,
        "overall_dump_ins_percentage": overall_dump_ins_percentage,

        "total_dump_outs_made": total_dump_outs_made,
        "average_dump_outs_made": average_dump_outs_made,
        "average_dump_outs_percentage": average_dump_outs_percentage,
        "overall_dump_outs_percentage": overall_dump_outs_percentage,

    }
    
    return player_passing_stats_dict

In [12]:
def zone_entry_stats(player_name, df):
    df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
    not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()

    df_zone_entry = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Zone Entry') & (x['Player']==player_name)]['Event'].count()).to_frame(name='zong_entry_number')
    df_zone_entry = df_zone_entry.drop(not_played_dates)

    num_games_played = len(df_zone_entry)

    df_zone_entry

    total_zone_entries_made = df_zone_entry["zong_entry_number"].sum()
    average_zone_entries_made = df_zone_entry["zong_entry_number"].mean()

    player_zone_entry_stats_dict = {
        "name": player_name,
        "num_games_played": num_games_played,
        "total_zone_entries_made": total_zone_entries_made,
        "average_zone_entries_made": average_zone_entries_made
    }

    return player_zone_entry_stats_dict

In [13]:
# passing_stats_list = list()
# shooting_stats_list = list()
# dumping_stats_list = list()
# zone_entry_stats_list = list()

# for player in player_list:
#     player_shooting_stats = shooting_stats(player, df_NWHL.copy())
#     shooting_stats_list.append(player_shooting_stats)
    
#     player_passing_stats = passing_stats(player, df_NWHL.copy())
#     passing_stats_list.append(player_passing_stats)
    
#     player_dumping_stats = dumping_stats(player, df_NWHL.copy())
#     dumping_stats_list.append(player_dumping_stats)
    
#     player_zone_entry_stats = zone_entry_stats(player, df_NWHL.copy())
#     zone_entry_stats_list.append(player_zone_entry_stats)


In [14]:
player_stats_summary = dict()

for player in player_list:
    player_shooting_stats = shooting_stats(player, df_NWHL.copy())
    player_passing_stats = passing_stats(player, df_NWHL.copy())
    player_dumping_stats = dumping_stats(player, df_NWHL.copy())
    player_zone_entry_stats = zone_entry_stats(player, df_NWHL.copy())
    
    stats_sum = player_shooting_stats | player_passing_stats | player_dumping_stats | player_zone_entry_stats
    player_stats_summary[player] = dict()
    for key, values in stats_sum.items():
        player_stats_summary[player][key] = values
    


C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\4012090281.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_number"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\3461801813.py:47: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"].sum())/(df_dumping["number_of_lost_dump_out"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\4012090281.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_number"].sum())/(df_scoring_merged["total_shot_number"

C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\4012090281.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_number"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_17908\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_du

In [15]:
player_stats_summary

{'Briana Mastel': {'name': 'Briana Mastel',
  'num_games_played': 5,
  'total_shots_made': 13,
  'total_goals_made': 0,
  'average_shots_made': 2.6,
  'average_shots_percentage': 0.0,
  'overall_shots_percentage': 0.0,
  'total_passes_made': 102,
  'total_plays_made': 69,
  'average_passes_made': 20.4,
  'average_passes_percentage': 0.6796050798130346,
  'overall_passes_percentage': 0.6764705882352942,
  'total_assists': 0,
  'total_dump_ins_made': 5,
  'average_dump_ins_made': 1.0,
  'average_dump_ins_percentage': 0.1111111111111111,
  'overall_dump_ins_percentage': 0.25,
  'total_dump_outs_made': 13,
  'average_dump_outs_made': 2.6,
  'average_dump_outs_percentage': 0.15,
  'overall_dump_outs_percentage': 0.18181818181818182,
  'total_zone_entries_made': 2,
  'average_zone_entries_made': 0.4},
 'Carlee Turner': {'name': 'Carlee Turner',
  'num_games_played': 7,
  'total_shots_made': 23,
  'total_goals_made': 0,
  'average_shots_made': 3.2857142857142856,
  'average_shots_percentage':

In [51]:
df_player_stats_summary = pd.DataFrame.from_dict(player_stats_summary).T

In [53]:
df_player_stats_summary.drop(["name"], axis=1, inplace=True)
df_player_stats_summary.replace(np.Inf, np.nan, inplace=True)

In [54]:
df_player_stats_summary

,num_games_played,total_shots_made,total_goals_made,average_shots_made,average_shots_percentage,overall_shots_percentage,total_passes_made,total_plays_made,average_passes_made,average_passes_percentage,...,total_dump_ins_made,average_dump_ins_made,average_dump_ins_percentage,overall_dump_ins_percentage,total_dump_outs_made,average_dump_outs_made,average_dump_outs_percentage,overall_dump_outs_percentage,total_zone_entries_made,average_zone_entries_made
Briana Mastel,5,13,0,2.600000,0.000000,0.000000,102,69,20.400000,0.679605,...,5,1.000000,0.111111,0.250000,13,2.600000,0.150000,0.181818,2,0.400000
Carlee Turner,7,23,0,3.285714,0.000000,0.000000,79,57,11.285714,0.714710,...,3,0.428571,0.000000,0.000000,16,2.285714,0.119048,0.142857,20,2.857143
Christina Putigna,7,39,2,5.571429,0.064286,0.051282,145,98,20.714286,0.679692,...,16,2.285714,0.214286,0.333333,4,0.571429,0.000000,0.000000,41,5.857143
Jenna Rheault,3,4,0,1.333333,0.000000,0.000000,31,21,10.333333,0.619565,...,2,0.666667,0.000000,0.000000,1,0.333333,0.000000,0.000000,5,1.666667
Jillian Dempsey,6,27,3,4.500000,0.103175,0.111111,116,80,19.333333,0.692628,...,3,0.500000,0.000000,0.000000,7,1.166667,0.000000,0.000000,19,3.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sarah Steele,6,17,0,2.833333,0.000000,0.000000,213,158,35.500000,0.733336,...,7,1.166667,0.000000,0.000000,4,0.666667,0.333333,0.333333,11,1.833333
Sarah-Eve Coutu Godbout,6,43,1,7.166667,0.020833,0.023256,94,56,15.666667,0.617356,...,13,2.166667,0.400000,0.625000,3,0.500000,0.000000,0.000000,34,5.666667
Shiann Darkangelo,6,36,0,6.000000,0.000000,0.000000,217,154,36.166667,0.706360,...,15,2.500000,0.111111,0.153846,9,1.500000,0.138889,0.285714,37,6.166667
Taylor Woods,6,46,4,7.666667,0.087572,0.086957,203,157,33.833333,0.775013,...,15,2.500000,0.027778,0.071429,16,2.666667,0.066667,0.066667,30,5.000000


In [55]:
df_player_stats_summary_normalized = df_player_stats_summary.copy()

In [56]:
metrics_list = df_player_stats_summary_normalized.columns.to_list()

In [57]:
for metric in metrics_list:
    max_val = df_player_stats_summary_normalized[metric].max()
    min_val = df_player_stats_summary_normalized[metric].min()
    df_player_stats_summary_normalized[metric] = df_player_stats_summary_normalized[metric].apply(lambda x: (x-min_val)/(max_val-min_val)*10)

In [58]:
df_player_stats_summary_normalized

,num_games_played,total_shots_made,total_goals_made,average_shots_made,average_shots_percentage,overall_shots_percentage,total_passes_made,total_plays_made,average_passes_made,average_passes_percentage,...,total_dump_ins_made,average_dump_ins_made,average_dump_ins_percentage,overall_dump_ins_percentage,total_dump_outs_made,average_dump_outs_made,average_dump_outs_percentage,overall_dump_outs_percentage,total_zone_entries_made,average_zone_entries_made
Briana Mastel,6.666667,2.241379,0.0,2.888889,0.000000,0.000000,3.051360,2.821577,4.178462,5.194076,...,2.272727,2.105263,1.111111,2.500000,5.652174,6.500000,1.500000,1.818182,0.322581,0.413793
Carlee Turner,10.000000,3.965517,0.0,3.650794,0.000000,0.000000,2.356495,2.323651,2.215385,5.720654,...,1.363636,0.902256,0.000000,0.000000,6.956522,5.714286,1.190476,1.428571,3.225806,2.955665
Christina Putigna,10.000000,6.724138,4.0,6.190476,1.285714,1.794872,4.350453,4.024896,4.246154,5.195377,...,7.272727,4.812030,2.142857,3.333333,1.739130,1.428571,0.000000,0.000000,6.612903,6.059113
Jenna Rheault,3.333333,0.689655,0.0,1.481481,0.000000,0.000000,0.906344,0.829876,2.010256,4.293478,...,0.909091,1.403509,0.000000,0.000000,0.434783,0.833333,0.000000,0.000000,0.806452,1.724138
Jillian Dempsey,8.333333,4.655172,6.0,5.000000,2.063492,3.888889,3.474320,3.278008,3.948718,5.389425,...,1.363636,1.052632,0.000000,0.000000,3.043478,2.916667,0.000000,0.000000,3.064516,3.275862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sarah Steele,8.333333,2.931034,0.0,3.148148,0.000000,0.000000,6.404834,6.514523,7.430769,6.000046,...,3.181818,2.456140,0.000000,0.000000,1.739130,1.666667,3.333333,3.333333,1.774194,1.896552
Sarah-Eve Coutu Godbout,8.333333,7.413793,2.0,7.962963,0.416667,0.813953,2.809668,2.282158,3.158974,4.260338,...,5.909091,4.561404,4.000000,6.250000,1.304348,1.250000,0.000000,0.000000,5.483871,5.862069
Shiann Darkangelo,8.333333,6.206897,0.0,6.666667,0.000000,0.000000,6.525680,6.348548,7.574359,5.595405,...,6.818182,5.263158,1.111111,1.538462,3.913043,3.750000,1.388889,2.857143,5.967742,6.379310
Taylor Woods,8.333333,7.931034,8.0,8.518519,1.751443,3.043478,6.102719,6.473029,7.071795,6.625196,...,6.818182,5.263158,0.277778,0.714286,6.956522,6.666667,0.666667,0.666667,4.838710,5.172414


In [60]:
df_player_stats_summary_normalized = df_player_stats_summary_normalized.round(2)

In [61]:
df_player_stats_summary_normalized.to_csv("df_player_stats_summary_normalized.csv")

In [90]:
df_test = df_NWHL.copy()

In [91]:
df_test['Clock'] = pd.to_datetime(df_test['Clock'], format="%M:%S")

In [96]:
(df_test.loc[0, "Clock"]-df_test.loc[1, "Clock"]).seconds

2